In [ ]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', 100)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from scipy import stats
from scipy.stats import norm, skew

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [ ]:
df_train = pd.read_csv('boston.csv')
print(df_train.shape)
print(df_train.columns)

In [ ]:
df_train.info()

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
# Split dataset into feature matrix X and target value y
X_train = df_train.drop('INDUS', axis=1)
y_train = df_train['INDUS']

In [ ]:
sns.displot(y_train, kde=True);

In [ ]:
# Check these attributes for target's distribution
print("Skewness: %f" % y_train.skew())
print("Kurtosis: %f" % y_train.kurt())

In [ ]:
# Correlation Matrix on Heatmap
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, square=True, cmap='RdBu_r');

In [ ]:
# Rank & Plot Top 9 Features Correlated with Target
k = 10 
cols = corrmat.nlargest(k, 'INDUS')['INDUS'].index
cm = np.corrcoef(df_train[cols].values.T)
f, ax = plt.subplots(figsize=(12, 9))
sns.set(font_scale=1.2)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values, cmap='RdBu_r')
plt.show()

In [ ]:
# Further, Top Correlated Features Can be Picked for Scatterplot to See if Some Linear Relations We Could Find  
cols = ['INDUS', 'NOX', 'TAX', 'AGE', 'LSTAT', 'RAD', 'CRIM']
sns.pairplot(df_train[cols])
plt.show()

In [ ]:
# First, Find & Sort NULL Values Existed in Features (Numerical)
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum() / df_train.isnull().count()).sort_values(ascending=False) # df_train.count() will count non-NA by default
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
# Here, We Drop All Features with > 1 NULL Values and Drop 1 NULL Item for Electrical
df_train = df_train.drop(missing_data[missing_data['Total'] > 1].index, 1)
df_train = df_train.drop(df_train.loc[df_train['INDUS'].isnull()].index)
df_train.isnull().sum().max()

In [ ]:
# We could find 2 outliers (right-bottom) are obviously out of this near-linear relation
fig, ax = plt.subplots()
ax.scatter(X_train['CRIM'], y_train)
plt.ylabel('INDUS', fontsize=15)
plt.xlabel('CRIM', fontsize=15)
plt.show()

In [ ]:
# Delete Outliers (crietrias could be obvsered and choosen, here > 4000 and < 300000)
df_train = df_train.drop(df_train[(df_train['INDUS'] > 4000) & (df_train['CRIM'] < 300000)].index)

# Plot De-Outlier Graph
fig, ax = plt.subplots()
ax.scatter(df_train['INDUS'], df_train['CRIM'])
plt.ylabel('CRIM', fontsize=15)
plt.xlabel('INDUS', fontsize=15)
plt.show()

In [ ]:
# After Plotting Distribution & Probability, skewness and peakness could be observered
sns.distplot(df_train['CRIM'] , fit = norm);
fig = plt.figure()
res = stats.probplot(df_train['CRIM'], plot = plt)

In [ ]:
# After Applying Log Transformation, Much More Fit on Normal Distribution
df_train['CRIM'] = np.log(df_train['CRIM'])
sns.distplot(df_train['CRIM'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['CRIM'], plot=plt)

In [ ]:
df_train = pd.get_dummies(df_train)
print(df_train.shape)

In [ ]:
# Lasso Regression
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [ ]:
# Elastic Net Regression
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [ ]:
# Kernel Ridge Regression
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

In [ ]:
# Gradient Boosting Regression
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [ ]:
# XGBoost
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [ ]:
# LightGBM
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [ ]:
# Define CV Score Function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(df_train.values)
    rmse= np.sqrt(-cross_val_score(model, df_train.values, df_train['CRIM'].values, scoring="neg_mean_squared_error", cv=kf))
    return(rmse)

In [ ]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [ ]:
!pip install nb2xls

In [ ]:
df_train.to_excel('boston_file.xlsx')